In [1]:
import torch
from pytorch_pretrained_bert import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel

In [46]:
import torch.nn as nn

In [48]:
from fastai.text.models import MultiHeadRelativeAttention

In [47]:
TransfoXLLMHeadModel??

In [2]:
model = TransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103')
model.eval()

100%|██████████| 606/606 [00:00<00:00, 463062.17B/s]


TransfoXLLMHeadModel(
  (transformer): TransfoXLModel(
    (word_emb): AdaptiveEmbedding(
      (emb_layers): ModuleList(
        (0): Embedding(20000, 1024)
        (1): Embedding(20000, 256)
        (2): Embedding(160000, 64)
        (3): Embedding(67735, 16)
      )
      (emb_projs): ParameterList(
          (0): Parameter containing: [torch.FloatTensor of size 1024x1024]
          (1): Parameter containing: [torch.FloatTensor of size 1024x256]
          (2): Parameter containing: [torch.FloatTensor of size 1024x64]
          (3): Parameter containing: [torch.FloatTensor of size 1024x16]
      )
    )
    (drop): Dropout(p=0.1)
    (layers): ModuleList(
      (0): RelPartialLearnableDecoderLayer(
        (dec_attn): RelPartialLearnableMultiHeadAttn(
          (qkv_net): Linear(in_features=1024, out_features=3072, bias=False)
          (drop): Dropout(p=0.1)
          (dropatt): Dropout(p=0.0)
          (o_net): Linear(in_features=1024, out_features=1024, bias=False)
          (laye

In [3]:
# Load pre-trained model tokenizer (vocabulary from wikitext 103)
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')

# Tokenized input
text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"
tokenized_text_1 = tokenizer.tokenize(text_1)
tokenized_text_2 = tokenizer.tokenize(text_2)

# Convert token to vocabulary indices
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)

# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])

100%|██████████| 9143613/9143613 [00:02<00:00, 4396097.96B/s]


In [44]:
# If you have a GPU, put everything on cuda
tokens_tensor_1 = tokens_tensor_1.to('cpu')
tokens_tensor_2 = tokens_tensor_2.to('cpu')
model.to('cpu')

with torch.no_grad():
    # Predict all tokens
    predictions_1, mems_1 = model(tokens_tensor_1)
    # We can re-use the memory cells in a subsequent call to attend a longer context
    predictions_2, mems_2 = model(tokens_tensor_2, mems=mems_1)

# get the predicted last token
predicted_index = list(map(lambda x:x.item(),predictions_2[0, :, :].argmax(1)))
predicted_token = tokenizer.convert_ids_to_tokens(predicted_index)


In [41]:
with torch.no_grad():
    hidden_2, mems_2 = model.transformer(tokens_tensor_2, mems=mems_1)

In [42]:
class MaskedKnowledgeAGG(torch.nn.module):
    def __init__(self,
                 vocab_sz:int,
                 d_model:int,
                 d_head:int,
                 n_heads:int,
                 embed_p:float=0.,
                 d_head_ent:int,
                 n_heads_ent:int,
                 embed_p_ent:float=0.,
                 d_ent:int,
                 mem_len:int,
                 n_layers:int,
                 mask:bool=True,
                 ent_vocab_sz:int,
                 d_inner:int
                 
                ):
        super().__init__()
        # get id as inputs and outputs vectors
        self.token_enc = nn.Embedding(vocab_sz, d_model)
        self.ent_enc = nn.Embedding(ent_vocab_sz, d_ent)
        self.pos_enc_tokens = nn.Embedding(ctx_len, d_model)
        self.pos_enc_ent = nn.Embedding(ctx_len, d_ent)
        self.drop_emb_token = nn.Dropout(embed_p)
        self.drop_emb_ent = nn.Dropout(embed_p_ent)

        self.u_tkn = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.u_ent = nn.Parameter(torch.Tensor(n_heads_ent, 1, d_head_ent)) #Remove 1 for einsum implementation of attention

        self.v_tkn = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.v_ent = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention

        self.mem_len,self.n_layers,self.d_model,self.mask = mem_len,n_layers,d_model,mask
        self.init = False
        self.ent_attention =[]
        self.tkn_attention = []
        for layer in layers:
            ent_attention.append(DecoderLayer(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls))
            


torch.Size([1, 5, 1024])

In [30]:
len(mems_2)

18

In [32]:
mems_2[0].shape

torch.Size([1600, 1, 1024])

In [43]:
predicted_token

['Henson', '?', 'in', 'man', 'who']

In [17]:
list(map(lambda x:x.item(),predictions_2[0, :, :].argmax(1)))

[12034, 788, 7, 435, 52]

In [11]:
predictions_2[0, -1, :]

267735

In [6]:
tokenizer.convert_ids_to_tokens([predicted_index])

['who']